In [64]:
# !apt-get install swig cmake
# #!pip install setuptools==65.5.0

# !pip install stable-baselines3[extra]

# !pip install imageio
# !pip install imageio_ffmpeg
# !pip install pyglet==1.5.1


In [65]:
# !apt update
# !apt install ffmpeg xvfb
# !pip install xvfbwrapper
# !pip install pyvirtualdisplay

In [66]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import A2C

from IPython import display
from IPython.display import HTML
import pygame
from base64 import b64encode
import matplotlib.pyplot as plt
import imageio
from time import sleep
import tqdm

import os

In [67]:
from stable_baselines3.common.evaluation import evaluate_policy

from stable_baselines3.common.callbacks import CheckpointCallback

In [68]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [69]:

# class LunarLanderRewardWrapper(gym.Wrapper):
#     def __init__(self, env):
#         super().__init__(env)

#     def step(self, action):
#         observation, reward, done, info = self.env.step(action)

#         # Supondo que a posição x do pouso está na posição de índice 0 de 'observation'
#         x_position = observation[0]

#         # Penalidade por não pousar no centro (você pode ajustar a fórmula conforme necessário)
#         penalty = np.abs(x_position) * 5  # Multipliquei por 5 para tornar a penalidade mais significativa

#         # Atualizar recompensa
#         reward -= penalty

#         return observation, reward, done, info

# # Criar o ambiente
# env = gym.make("LunarLander-v2")
# env = LunarLanderRewardWrapper(env)

In [70]:
env = gym.make("LunarLander-v2")

In [71]:
#gradient_steps - How many gradient steps to do after each rollout (see train_freq) Set to -1 means to do as many gradient steps as steps done in the environment during the rollout.

model = A2C(
    'MlpPolicy',
    env,
    n_steps=7,
    gamma=0.69,
    learning_rate=0.001,
    verbose=1
    )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [72]:
# Separate env for evaluation
eval_env = gym.make("LunarLander-v2")

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-137.46 +/- 68.81675790960028


In [73]:

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="./logs/",
  name_prefix="rl_model",
  save_replay_buffer=True,
  save_vecnormalize=True,
)

# Train the agent
model.learn(total_timesteps=50000, callback = checkpoint_callback)
# Save the agent
model.save("a2c_lunarlander")
#del model  # delete trained model to demonstrate loading

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 127      |
|    ep_rew_mean        | -506     |
| time/                 |          |
|    fps                | 428      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 700      |
| train/                |          |
|    entropy_loss       | -1.3     |
|    explained_variance | -0.011   |
|    learning_rate      | 0.001    |
|    n_updates          | 99       |
|    policy_loss        | -4.03    |
|    value_loss         | 26.8     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 127      |
|    ep_rew_mean        | -337     |
| time/                 |          |
|    fps                | 426      |
|    iterations         | 200      |
|    time_elapsed       | 3        |
|    total_timesteps    | 1400     |
| train/                |          |
|

In [74]:
model = A2C.load("a2c_lunarlander")


In [75]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-28.55 +/- 164.53278694190737


In [76]:
def convert_to_video(images, video_path):
  kargs = { 'macro_block_size': 1 }
  imageio.mimsave(video_path, [np.array(img) for i, img in enumerate(images)], fps=15, **kargs)

In [77]:
env = gym.make('LunarLander-v2', render_mode = "rgb_array")
obs = env.reset()
images = []
obs = obs[0]
for i in range(2000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    img = env.render()
    images.append(img)
    # VecEnv resets automatically
    if done:
      obs = env.reset()
      obs = obs[0]

In [78]:
video_path =  "replay.mp4"
convert_to_video(images, video_path)

In [79]:
# Show video
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)